In [1]:
import os
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# %matplotlib inline

# plt.rcParams['lines.linewidth'] = 0.8
# plt.rcParams['lines.markersize'] = 4.0
# plt.rcParams['figure.figsize'] = 20, 10

In [2]:
"""
データの順番 (trvl_csv)
/home/icer/Videos/Webcam/processed_data/nod2-2.csv
/home/icer/Videos/Webcam/processed_data/other2-2.csv
/home/icer/Videos/Webcam/processed_data/nod2-1.csv
/home/icer/Videos/Webcam/processed_data/other2-1.csv
/home/icer/Videos/Webcam/processed_data/other2-3.csv
/home/icer/Videos/Webcam/processed_data/nod2-3.csv
"""

trvl_csv_path = "/home/icer/Project/icer/openface/data/preprocess_araki.csv"
test_csv_dir = "/home/icer/Project/openface_dir/output_vid_dir/feature_splited_test_csv/"
test_csv_paths = [test_csv_dir + "multi_people_0.csv",
                  test_csv_dir + "multi_people_1.csv",
                  test_csv_dir + "multi_people_2.csv",
                  test_csv_dir + "multi_people_3.csv",
                 ]
trvl_df = pd.read_csv(trvl_csv_path, engine='python')
test_dfs = [pd.read_csv(test_csv_path, engine='python') for test_csv_path in test_csv_paths]

In [3]:
print("== train/validデータの詳細 ==")
trvl_df['label'].astype(int)
cl0 = (trvl_df['label'] == 0)
cl1 = (trvl_df['label'] == 1)
print(f"cl0: {cl0.sum()}, cl1: {cl1.sum()}")
print(f"cl0: {cl0.sum()/(cl0.sum() + cl1.sum())}, cl1: {cl1.sum()/(cl0.sum() + cl1.sum())}")

trvl_df

== train/validデータの詳細 ==
cl0: 239, cl1: 160
cl0: 0.5989974937343359, cl1: 0.40100250626566414


,Unnamed: 0,mean_x,std_x,mad_x,max_x,min_x,energy_x,entropy_x,iqr_x,range_x,...,entropy_z,iqr_z,range_z,skewness_z,kurtosis_z,frame_in,frame_out,timestamp_in,timestamp_out,label
0,0,0.006844,0.050951,0.0335,0.068,-0.114,0.002643,-inf,0.08425,0.182,...,-inf,0.09425,0.154,0.288801,-1.388185,1.0,32.0,0.000,1.033,1.0
1,1,0.012406,0.126853,0.0690,0.348,-0.166,0.016246,-inf,0.11450,0.514,...,-inf,0.04250,0.117,-0.033182,-0.791665,17.0,48.0,0.533,1.567,1.0
2,2,0.088375,0.143556,0.1005,0.348,-0.166,0.028418,-inf,0.20050,0.514,...,-inf,0.03100,0.089,-0.748987,-0.033459,33.0,64.0,1.067,2.100,1.0
3,3,0.088625,0.175553,0.0895,0.360,-0.421,0.038673,-inf,0.17525,0.781,...,-inf,0.01900,0.072,-0.845825,0.957004,49.0,80.0,1.600,2.633,1.0
4,4,0.107312,0.188037,0.1060,0.360,-0.421,0.046874,-inf,0.15250,0.781,...,-inf,0.03250,0.069,0.071124,-0.966064,65.0,96.0,2.133,3.167,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,394,-0.051437,0.062537,0.0375,0.102,-0.147,0.006557,-inf,0.06225,0.249,...,3.428758,0.02450,0.063,0.092663,-0.683991,961.0,992.0,32.000,33.033,1.0
395,395,-0.015906,0.165907,0.0695,0.429,-0.191,0.027778,-inf,0.15575,0.620,...,3.250046,0.06375,0.233,-1.315751,0.350949,977.0,1008.0,32.533,33.567,1.0
396,396,0.085000,0.199447,0.1515,0.429,-0.191,0.047004,-inf,0.33050,0.620,...,3.277366,0.16125,0.222,-0.281449,-1.502605,993.0,1024.0,33.067,34.100,1.0
397,397,0.152594,0.163435,0.1570,0.429,-0.080,0.049996,-inf,0.31725,0.509,...,3.341211,0.12875,0.221,0.062897,-1.278214,1009.0,1040.0,33.600,34.633,1.0


## データの前処理

In [4]:
def drop_miss_values(df, axis=0):
    """
    欠損値のある行を削除
    """
    df = df.replace([np.inf, -np.inf], np.nan)
    print(f"欠損値:\n{df.isnull().sum()}")
    dropped_df = df.dropna(axis=axis, how="any")
    return dropped_df


def feature_selection(X, y=None, selected_bool_list: list=None):
    """
    分類に有用な特徴量を選択
    """
    from sklearn.ensemble import RandomForestRegressor
    from boruta import BorutaPy
    
    if (selected_bool_list is None) and (y is not None):
        # RandomForestRegressorでBorutaを実行
        rf = RandomForestRegressor(n_jobs=-1, max_depth=5)
        feat_selector = BorutaPy(rf, n_estimators='auto', verbose=0, random_state=1)
        feat_selector.fit(X.values, y.values)

        # 選択された特徴量を確認
        selected_bool_list = feat_selector.support_
        print('選択された特徴量の数: %d' % np.sum(selected_bool_list))
        print(selected_bool_list)
        print(X.columns[selected_bool_list])

    X_selected = X.iloc[:, selected_bool_list]
    return X_selected, selected_bool_list


def get_Xy_from_df(df):
    X = df.iloc[:, 1:-5]
    y = df['label'].astype(int)
    return X, y

## モデルの訓練

In [5]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler 
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

random_state = 0

# train/validデータの前処理
trvl_df = drop_miss_values(trvl_df)  # 欠損値削除
X_trvl, y_trvl = get_Xy_from_df(trvl_df)  # 特徴量, ラベルデータを取得
X_trvl, selected_bool_list = feature_selection(X=X_trvl, y=y_trvl)  # 特徴量を選択

# データの標準化処理
sc = StandardScaler()
sc.fit(X_trvl)
X_trvl = sc.transform(X_trvl)

# trainとvalid用に分割
X_train, X_valid = X_trvl[142:], X_trvl[0:142]
y_train, y_valid = y_trvl[142:], y_trvl[0:142]


# kernel SVMのインスタンスを生成
model = SVC(kernel='rbf', random_state=random_state)

# モデルの学習
model.fit(X_train, y_train)

# trainデータに対する評価
pred_train = model.predict(X_train)
accuracy_train = accuracy_score(y_train, pred_train)
precision_train = precision_score(y_train, pred_train)
recall_train = recall_score(y_train, pred_train)
print('トレーニングデータに対する正解率： %.2f' % accuracy_train)
print('トレーニングデータに対する適合率： %.2f' % precision_train)
print('トレーニングデータに対する再現率： %.2f' % recall_train)
cm = confusion_matrix(y_train, pred_train)
print(cm)

欠損値:
Unnamed: 0         0
mean_x             0
std_x              0
mad_x              0
max_x              0
min_x              0
energy_x           0
entropy_x        140
iqr_x              0
range_x            0
skewness_x         0
kurtosis_x         0
mean_y             0
std_y              0
mad_y              0
max_y              0
min_y              0
energy_y           0
entropy_y         31
iqr_y              0
range_y            0
skewness_y         0
kurtosis_y         0
mean_z             0
std_z              0
mad_z              0
max_z              0
min_z              0
energy_z           0
entropy_z         90
iqr_z              0
range_z            0
skewness_z         0
kurtosis_z         0
frame_in           0
frame_out          0
timestamp_in       0
timestamp_out      0
label              0
dtype: int64
選択された特徴量の数: 13
[ True  True  True  True False  True  True  True  True  True False False
 False False False False False False False False False False  True False
 F

## validデータで評価

In [6]:
# validデータに対する評価
pred_valid = model.predict(X_valid)
accuracy_valid = accuracy_score(y_valid, pred_valid)
precision_valid = precision_score(y_valid, pred_valid)
recall_valid = recall_score(y_valid, pred_valid)
print(f"validデータに対する正解率： {accuracy_valid: .2f}")
print(f"validデータに対する適合率： {precision_valid: .2f}")
print(f"validデータに対する再現率： {recall_valid: .2f}")

cm = confusion_matrix(y_valid, pred_valid)
print(cm)

validデータに対する正解率：  0.98
validデータに対する適合率：  0.95
validデータに対する再現率：  0.90
[[120   1]
 [  2  19]]


## テスト

In [7]:
def output_csv(
    base_df: pd.core.frame.DataFrame,
    pred_result: np.ndarray,
    output_path: str="output_pred.csv",
    mode: str="test"
):
    """
    base_df: predに対応するDataFrame
    pred_result: 判別後の予測結果 (ndarray)
    """
    if mode == "test":
        in_columns = ['frame_in', 'frame_out']
        out_columns = ['frame', 'pred']
    else:
        in_columns = ['frame_in', 'frame_out', 'label']
        out_columns = ['frame', 'label', 'pred']
    
    base_df = base_df[in_columns]
    base_df = base_df.assign(pred=pred_result)
    
    frame_in = base_df['frame_in'].iloc[0].astype(int)
    frame_out = base_df['frame_out'].iloc[-1].astype(int)
    cum_pred = np.zeros(frame_out - frame_in)
    


    out_df = pd.DataFrame(columns=out_columns)

    for i, (_, row) in enumerate(base_df.iterrows()):
        fin = row['frame_in'].astype(int)
        fout = row['frame_out'].astype(int)

        for frame in range(fin, fout + 1):
            if (out_df['frame'] == frame).sum() == 0:
                if mode == "test":
                    out_df = out_df.append({
                        'frame': frame,
                        'pred': row['pred'],
                    }, ignore_index=True)
                else:
                    out_df = out_df.append({
                        'frame': frame,
                        'label': row['label'],
                        'pred': row['pred'],
                    }, ignore_index=True)
            else:
                if row['pred'] == 1:
                    out_df.loc[out_df['frame'] == frame, 'pred'] += 1
                    
    if output_path is not None:
        out_df.to_csv(output_path, index=False)

    return out_df

In [11]:
for i, test_df in enumerate(test_dfs):
    # テストデータに対する前処理
    test_df = drop_miss_values(test_df)
    X_test = test_df.iloc[:, 1:-4]  # 特徴量を取得
    print(len(selected_bool_list))
    print(len(trvl_df.columns))
    print(len(test_df.columns))
    X_test, _ = feature_selection(X=X_test, selected_bool_list=selected_bool_list)
    X_test = sc.transform(X_test)

    # テストデータに対する精度
    pred_test = model.predict(X_test)    
    output_csv(test_df, pred_test, output_path=f"pred_multi_id{i}.csv")

欠損値:
Unnamed: 0        0
mean_x            0
std_x             0
mad_x             0
max_x             0
min_x             0
energy_x          0
entropy_x        12
iqr_x             0
range_x           0
skewness_x        0
kurtosis_x        0
mean_y            0
std_y             0
mad_y             0
max_y             0
min_y             0
energy_y          0
entropy_y         4
iqr_y             0
range_y           0
skewness_y        0
kurtosis_y        0
mean_z            0
std_z             0
mad_z             0
max_z             0
min_z             0
energy_z          0
entropy_z         8
iqr_z             0
range_z           0
skewness_z        0
kurtosis_z        0
frame_in          0
frame_out         0
timestamp_in      0
timestamp_out     0
dtype: int64
33
39
38
欠損値:
Unnamed: 0        0
mean_x            0
std_x             0
mad_x             0
max_x             0
min_x             0
energy_x          0
entropy_x        15
iqr_x             0
range_x           0
skewness

## 実行結果をビデオに反映

In [12]:
import cv2
import tqdm
from typing import Tuple


def get_video_capture(video_path: str) -> cv2.VideoCapture:
    return cv2.VideoCapture(video_path)


def get_video_length(video_path: str) -> int:
    video_capture = get_video_capture(video_path)
    return int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))


def get_frame_position(video_capture: cv2.VideoCapture) -> int:
    return int(video_capture.get(cv2.CAP_PROP_POS_FRAMES))


def set_frame_position(video_capture: cv2.VideoCapture, position: int) -> int:
    return int(video_capture.set(cv2.CAP_PROP_POS_FRAMES, position))


def get_video_dimension(video_path: str) -> Tuple[int, int]:
    video_capture = get_video_capture(video_path)
    return int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))


def get_video_framerate(video_path: str) -> float:
    video_capture = get_video_capture(video_path)
    return float(video_capture.get(cv2.CAP_PROP_FPS))


def output_video_nod(nod_df, face_df, video_path: str, output_path: str = "output_video_nod"):
    video_capture = get_video_capture(video_path)
    video_length = get_video_length(video_path)

    print(nod_df)
    start, end = nod_df['frame'].iloc[0].astype(int), nod_df['frame'].iloc[-1].astype(int)

    fourcc = cv2.VideoWriter_fourcc(*"XVID")
    video_writer = cv2.VideoWriter(output_path, fourcc, get_video_framerate(video_path),
                                   get_video_dimension(video_path))

    frame_index = start
    set_frame_position(video_capture, start)  # Move position
    while video_capture.isOpened() and get_frame_position(video_capture) in range(start, end + 1):
        ret, frame = video_capture.read()
        
        nod = nod_df[nod_df['frame'] == frame_index]['pred'].values[0]
        texts = {0: "No", 1: "Nod", 2: "Nod!!!"}

        # Draw circle when nod flag == 1 or 2
        if nod > 0:
            face_df_frame = face_df.iloc[frame_index]
            mid = (int(face_df_frame[' x_30']), int(face_df_frame[' y_30']))
            radius = int(abs(face_df_frame[' y_8'] - face_df_frame[' y_30']))
#             frame = cv2.circle(frame, mid, radius=radius, color=(0, 0, 255), thickness=int(1 * nod))

            text = texts[nod]
            font = cv2.FONT_HERSHEY_PLAIN
            textsize = cv2.getTextSize(text, font, 1, 2)[0]
            coord = (mid[0] - int(textsize[0] * 1.7), mid[1] - int(radius * 2))
            frame = cv2.putText(frame, text, coord, font, 4, (0, 0, 255), 5, cv2.LINE_AA)

        video_writer.write(frame)
        frame_index += 1

    video_capture.release()
    video_writer.release()


video_path = "/home/icer/Project/openface_dir/output_vid_dir/test.mp4"
# output_path = "pred_test.mp4"
for i, face_csv_path in enumerate(test_csv_paths):
    nod_df = pd.read_csv(f"pred_multi_id{i}.csv")
    face_df = pd.read_csv(face_csv_path)
    output_path = f"pred_test_id{i}.mp4"
    output_video_nod(nod_df, face_df, video_path, output_path)

     frame  pred
0     17.0   0.0
1     18.0   0.0
2     19.0   0.0
3     20.0   0.0
4     21.0   0.0
..     ...   ...
219  486.0   0.0
220  487.0   0.0
221  488.0   0.0
222  489.0   0.0
223  490.0   0.0

[224 rows x 2 columns]


IndexError: index 0 is out of bounds for axis 0 with size 0